In [ ]:
! pip install kaggle
! kaggle datasets download puneet6060/intel-image-classification
! unzip -n -d dataset intel-image-classification.zip

# Imports

- numpy

In [ ]:
! pip install numpy
! pip install pillow
! pip install matplotlib
! pip install scikit-learn

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
training_path = "./dataset/seg_train/seg_train"
validation_path = "./dataset/seg_test/seg_test"
training_subdirs = os.listdir(training_path)
validation_subdirs = os.listdir(validation_path)

training_images = [os.path.join(os.path.join(training_path, dir), (os.listdir(os.path.join(training_path, dir))[0])) for dir in os.listdir(training_path)]
validation_images = [os.path.join(os.path.join(validation_path, dir), (os.listdir(os.path.join(validation_path, dir))[0])) for dir in os.listdir(validation_path)]
training_images = [Image.open(image) for image in training_images]
validation_images = [Image.open(image) for image in validation_images]
fig, axs = plt.subplots(nrows=4, ncols=3)
for i, ax in enumerate(axs):
    for j, slot in enumerate(ax):
        slot.axis('off')
        if i < 2:
            slot.imshow(training_images[i*len(ax) + j])
            slot.set_title(f"{training_subdirs[i*len(ax) + j]}")
        else:
            slot.imshow(validation_images[(i-2)*len(ax) + j])
            slot.set_title(f"{validation_subdirs[(i-2)*len(ax) + j]}")
fig.tight_layout()
plt.show()

In [ ]:
train = tf.keras.utils.image_dataset_from_directory(training_path,
                                                      labels='inferred',
                                                      label_mode='categorical',
                                                      batch_size=32,
                                                      image_size=(150, 150))
valid = tf.keras.utils.image_dataset_from_directory(validation_path,
                                                      labels='inferred',
                                                      label_mode='categorical',
                                                      batch_size=32,
                                                      image_size=(150, 150))
class_names = train.class_names
class_names

In [ ]:
for i in train.as_numpy_iterator():
    plt.axis('off')
    plt.imshow(i[0][0].astype('uint8'))
    plt.show()
    break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train = train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
valid = valid.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal", input_shape=(150,150,3)),
                                         tf.keras.layers.RandomRotation(0.01), tf.keras.layers.RandomZoom(0.1)])

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Embedding
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers

model = Sequential([
  data_augmentation,
  layers.Rescaling(1.0/255.0, input_shape=(150,150,3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(512, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(6, name="outputs", activation='softmax')
])

print(model.summary())

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
patience=10
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='model.hdf5', save_best_only=True, verbose=1)
tb = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=1)

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=5e-3, momentum=5e-3), metrics=['accuracy'])

network_history = model.fit(train, validation_data=valid, epochs=50, verbose=1, callbacks=[early_stopping, checkpointer, tb])

In [ ]:
acc = network_history.history['accuracy']
val_acc = network_history.history['val_accuracy']

loss = network_history.history['loss']
val_loss = network_history.history['val_loss']

epochs_range = range(48)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model = tf.keras.models.load_model("/kaggle/working/model.hdf5")
X_test = "/kaggle/input/intel-image-classification/seg_pred/seg_pred/10090.jpg"

img = tf.keras.utils.load_img(
    X_test, target_size=(150, 150)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)